In [6]:
!pip install sqlalchemy
!pip install pyodbc
## for MSSQL ONLY
##!pip install pymssql  

In [17]:
from sqlalchemy import create_engine, MetaData, Table, inspect
import pandas as pd
import os
import sys

In [62]:

def db_conn(dialect, server, database, user = None, passw = None, sa = True):
    if sa:
        #SQL server auth
        assert user, 'You need a user name'
        assert passw, 'You need a password'
        conn = f"{dialect}://{user}:{passw}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"
    else:
        #Windows auth
        conn = f"{dialect}://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server"
    ##print(conn)
    return create_engine(conn)

In [63]:
import db_conf

# Setup the connection
dbconn = db_conn('mssql', db_conf.SERVER+db_conf.HOST, db_conf.DB, db_conf.USER, db_conf.PASSWORD, sa=True)


In [64]:
connection = dbconn.connect()
print(type(connection))

<class 'sqlalchemy.engine.base.Connection'>


In [65]:
# Save the table names to a list:
insp = inspect(dbconn)
schemas = insp.get_schema_names()
tables = insp.get_table_names()
print(tables)
print(schemas)
print('\n')

for schema in schemas:
    print(schema)

['AWBuildVersion', 'DatabaseLog', 'ErrorLog']
['db_accessadmin', 'db_backupoperator', 'db_datareader', 'db_datawriter', 'db_ddladmin', 'db_denydatareader', 'db_denydatawriter', 'db_owner', 'db_securityadmin', 'dbo', 'guest', 'HumanResources', 'INFORMATION_SCHEMA', 'Person', 'Production', 'Purchasing', 'Sales', 'sys']


db_accessadmin
db_backupoperator
db_datareader
db_datawriter
db_ddladmin
db_denydatareader
db_denydatawriter
db_owner
db_securityadmin
dbo
guest
HumanResources
INFORMATION_SCHEMA
Person
Production
Purchasing
Sales
sys


In [66]:
for col in insp.get_columns(table_name='Customer',schema='Sales'):
    #print(col)
    print(col['name'])

CustomerID
PersonID
StoreID
TerritoryID
AccountNumber
rowguid
ModifiedDate


In [67]:
## QUERY THE DB
# get a dataframe
df = pd.read_sql(sql = "SELECT * from Sales.Customer", con = connection)
df.head(10)

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,1,NaN,934.0,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263
1,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
2,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
3,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
4,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
5,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263
6,7,NaN,930.0,1,AW00000007,03E9273E-B193-448E-9823-FE0C44AEED78,2014-09-12 11:15:07.263
7,8,NaN,1024.0,5,AW00000008,801368B1-4323-4BFA-8BEA-5B5B1E4BD4A0,2014-09-12 11:15:07.263
8,9,NaN,620.0,5,AW00000009,B900BB7F-23C3-481D-80DA-C49A5BD6F772,2014-09-12 11:15:07.263
9,10,NaN,928.0,6,AW00000010,CDB6698D-2FF1-4FBA-8F22-60AD1D11DABD,2014-09-12 11:15:07.263


In [68]:
## JOINING query
multijoin = """
    SELECT
        A.CustomerID,
        A.StoreID,
        A.AccountNumber,
        B.SalesOrderID,
        C.ProductID
    FROM
        Sales.Customer AS A INNER JOIN Sales.SalesOrderHeader AS B
        ON A.CustomerID = b.CustomerID
    INNER JOIN
        Sales.SalesOrderDetail AS C
        ON B.SalesOrderID = C.SalesOrderID
"""
join_df = pd.read_sql(sql = multijoin, con = connection)
join_df.head()

,CustomerID,StoreID,AccountNumber,SalesOrderID,ProductID
0,29580,518.0,AW00029580,43665,707
1,29614,592.0,AW00029614,43668,707
2,29844,1086.0,AW00029844,43673,707
3,29824,1044.0,AW00029824,43677,707
4,29889,1184.0,AW00029889,43678,707


In [69]:
## Read a View
for v in insp.get_view_names(schema='Sales'):
    print(v)


vIndividualCustomer
vPersonDemographics
VProductQtySold
vSalesPerson
vSalesPersonSalesByFiscalYears
vStoreWithAddresses
vStoreWithContacts
vStoreWithDemographics


In [70]:
view = """
    SELECT * 
    FROM Sales.vSalesPersonSalesByFiscalYears
"""
show_view = pd.read_sql(sql = view, con = connection)
show_view.head(20)

,SalesPersonID,FullName,JobTitle,SalesTerritory,2002,2003,2004
0,275,Michael G Blythe,Sales Representative,Northeast,None,None,None
1,276,Linda C Mitchell,Sales Representative,Southwest,None,None,None
2,277,Jillian Carson,Sales Representative,Central,None,None,None
3,278,Garrett R Vargas,Sales Representative,Canada,None,None,None
4,279,Tsvi Michael Reiter,Sales Representative,Southeast,None,None,None
5,280,Pamela O Ansman-Wolfe,Sales Representative,Northwest,None,None,None
6,281,Shu K Ito,Sales Representative,Southwest,None,None,None
7,282,José Edvaldo Saraiva,Sales Representative,Canada,None,None,None
8,283,David R Campbell,Sales Representative,Northwest,None,None,None
9,284,Tete A Mensa-Annan,Sales Representative,Northwest,None,None,None


In [77]:
## Using CTE  WHERE Gender = 'M'
cte = """
WITH CTE_Employee AS (
    SELECT
        A.BusinessEntityID,
        A.FirstName,
        A.LastName,
        B.JobTitle
    FROM
        HumanResources.Employee AS B INNER JOIN Person.Person AS A
        ON B.BusinessEntityID = A.BusinessEntityID
)
SELECT
    CTE_Employee.BusinessEntityID,
    CTE_Employee.FirstName,
    CTE_Employee.LastName,
    CTE_Employee.JobTitle,
    ROUND(SUM(Sales.SubTotal),2) TotalSum 
FROM 
    CTE_Employee INNER JOIN Sales.SalesOrderHeader AS Sales
    ON CTE_Employee.BusinessEntityID = Sales.SalesPersonID
GROUP BY
    CTE_Employee.BusinessEntityID,
    CTE_Employee.FirstName,
    CTE_Employee.LastName,
    CTE_Employee.JobTitle
ORDER BY
    CTE_Employee.BusinessEntityID
"""

result = pd.read_sql(sql = cte, con = connection)
result.head(20)

,BusinessEntityID,FirstName,LastName,JobTitle,TotalSum
0,274,Stephen,Jiang,North American Sales Manager,1092123.86
1,275,Michael,Blythe,Sales Representative,9293903.00
2,276,Linda,Mitchell,Sales Representative,10367007.43
3,277,Jillian,Carson,Sales Representative,10065803.54
4,278,Garrett,Vargas,Sales Representative,3609447.21
5,279,Tsvi,Reiter,Sales Representative,7171012.75
6,280,Pamela,Ansman-Wolfe,Sales Representative,3325102.59
7,281,Shu,Ito,Sales Representative,6427005.55
8,282,José,Saraiva,Sales Representative,5926418.36
9,283,David,Campbell,Sales Representative,3729945.35


In [79]:
## CREATE A NEW TABLE

newdata = {
    'Sketch':['Cheese Shop','Silly Walks','Spanish Inquisition','Lumberjack Song','Argument Clinic'],
    'Length':['5:29','4:05','8:17','2:41','6:22']
    }
df2 = pd.DataFrame(data = newdata)
df2

# df2.to_sql(name = 'Monty Python', con = connection, schema = 'Sales', index = False, if_exists= 'replace')

# Read table - 
# pd.read_sql("SELECT * FROM Sales.MontyPython", connection)
# Delete table
from pandas.io import sql
# sql.execute('DROP TABELE IF EXISTS Sales.MontyPython', connection)

,Sketch,Length
0,Cheese Shop,5:29
1,Silly Walks,4:05
2,Spanish Inquisition,8:17
3,Lumberjack Song,2:41
4,Argument Clinic,6:22


In [ ]:
# Read: reading in data from the database into a Pandas DataFrame
# Products ranked based on sold amount between 2011-07-01 and 2011-08-01
example_sql ="""
SELECT 
  DENSE_RANK() OVER (ORDER BY SUM(SOD.OrderQty) DESC) AS SalesRank,
  P.ProductID,
  P.Name,
  SUM(SOD.OrderQty) AS TotalSold
FROM 
  Production.Product AS P 
  JOIN Sales.SalesOrderDetail AS SOD ON P.ProductID = SOD.ProductID 
  JOIN Sales.SalesOrderHeader AS SOH ON SOD.SalesOrderID = SOH.SalesOrderID
WHERE 
  SOH.OrderDate BETWEEN '2011-07-01' AND '2011-07-31'
GROUP BY
  P.ProductID, P.Name
ORDER BY
  SUM(SOD.OrderQty) DESC
"""

example_df = pd.read_sql(example_sql, connection, index_col='SalesRank')
example_df

In [ ]:
# Update: modifying the data in the DataFrame
# Adding a Goal column to th DataFrame, with the next month (August) sales goal of +10% TotalSold
example_df['August_Goal'] = round(example_df['TotalSold'] * 1.1, 0).astype(int)
example_df

# Save: saving the modified DataFrame into a new database table
example_df.to_sql(name='August_Goal_Table', con=connection, schema='Sales', index=False, if_exists='replace')

In [ ]:
# Double checking if the new table was created
check = pd.read_sql('SELECT * FROM Sales.August_Goal_Table', engine)
check